This takes way too long. Bigger buffer?

In [1]:
import json
import sqlite3
import numpy as np
import base64 

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

create_embeddings = """
    CREATE TABLE embeddings (
        id INTEGER PRIMARY KEY,         
        vector BLOB              
    );
"""
cur.execute(create_embeddings)
conn.commit()

insert_embeddings = """
    INSERT INTO embeddings (id, vector) VALUES (?, ?);
"""

buffer = []
with open('data/embeddings.jsonl', 'rt') as file:
    for line in file:
        j = json.loads(line)
        if not j['embedding']:
            j['embedding'] = ''
        buffer.append((
            j['id'], 
            np.frombuffer(base64.b64decode(j['embedding'])).tobytes()
        ))
        if len(buffer) > 100000:
            print("Writing buffer.")
            cur.executemany(
                insert_embeddings, 
                buffer
            )
            conn.commit()
            buffer = []

if len(buffer) > 0:
    print("Writing buffer.")        
    cur.executemany(
        insert_embeddings, 
        buffer
    )
    conn.commit()
    buffer = []

Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.
Writing buffer.


# Connect counts, clusters, and clean text.

In [2]:
from select_top_counts import select_top_counts

selected = select_top_counts()

# Drop duplicates
selected = selected.sort_values('count', ascending=False)
selected = selected.drop_duplicates('lower_title', keep='first')

In [3]:
import json 
import pandas as pd

clean = []
with open('data/cleaned.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        clean.append(json.loads(line))
clean = pd.DataFrame(clean)

# Drop duplicates
clean['len'] = clean.clean_text.apply(len)
clean = clean.sort_values('len', ascending=False)
clean = clean.drop_duplicates('lower_title', keep='first')
clean = clean.drop('len', axis=1)

In [4]:
clean['id'] = clean['id'].astype(np.int64)

In [5]:
clusters = []
with open('data/clusters.jsonl', 'r') as file:
    for line in file:
        clusters.append(json.loads(line))
clusters = pd.DataFrame(clusters)

In [6]:
print(len(clean))
clean = (clean
    .merge(selected, on='lower_title')
    .merge(clusters, on='id')
)
print(len(clean))

550751
548767


In [7]:
clean.sample()

,id,title,lower_title,clean_text,count,cluster
93506,700793,Complicated (Avril Lavigne song),complicated (avril lavigne song),"Complicated (Avril Lavigne song): ""'''Complica...",14,122


In [8]:
import sqlite3
import pandas as pd

try:
    conn = sqlite3.Connection('data/data.db')
    (clean
        .rename(columns={'lower_title': 'clean_title'})
        .drop(['clean_text'], axis=1)
        .to_sql('articles', conn)
    )
    conn.commit()
except Exception as e:
    print(e)
finally:
    conn.close()

In [9]:
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data/data.db')
cur = conn.cursor()

cur.execute("CREATE INDEX articles_idx_index ON articles(id)")
conn.commit()

cur.execute("CREATE INDEX embeddings_idx_index ON embeddings(id)")
conn.commit()